In [33]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get
qid_lab_get(42, 'en').keys()

dict_keys(['douglas adams'])

In [88]:
# wiki lang to iso
WIKI_DIR = '/backup/wikidata'
api_key = open('airtable_key.txt')
from pyairtable import Table, Api
import sys
sys.path.insert(0, '/projekti/nelma')
from mondoDB.referencedb import provi

if True:
    lang2cc = defaultdict(Counter)
    for k, v in provi.items():
        lang, s, c = v['id'].split('_')
        bod = float(v['Percent'])*3 if 'Percent' in v else 0
        if 'Status' in v:
            if v['Status'] == 'official':
                bod += 1
            elif v['Status'] == 'official_regional':
                bod += 0.5
            elif v['Status'] == 'de_facto_official':
                bod += 0.9
            elif v['Status'] == 'romanized':
                bod += 0.2
            elif v['Status'] == 'foreign':
                bod += -0.5
        if bod>0:
            lang2cc[f'{lang}'][c] = max(bod, lang2cc[f'{lang}'][c])
    api = Api(api_key)
    w2iso = {t['fields']['WMF']: t['fields']['qid'] if 'qid' in t['fields'] else None for t in Table(api_key, 'appUZvAm9EHZgC1Eg', 'wiki_lang').all()}
    wiki2cc = defaultdict(Counter)
    for k, v in w2iso.items():
        if len(k) == 2:
            wiki2cc[v].update(lang2cc[k])
        elif k.count('-') == 1:
            a, b = k.split('-')
            if len(a) == 2 and len(b) == 2:
                wiki2cc[v][b.upper()] += 5 
            elif len(a) == 2 and len(b) == 4:
                #wiki2cc[v].update(lang2cc[f'{k}_{b.uppser()}'])
                wiki2cc[v].update(lang2cc[a])
    wikil2cc = {k: {cc: round(100*v2/max(v.values())) for cc, v2 in v.items()} for k, v in wiki2cc.items() if len(v)>0 and max(v.values())>0}
    json.dump(wikil2cc, open(f'{WIKI_DIR}/wikil2cc.json', 'w'))
else:
    wikil2cc = json.load( open(f'{WIKI_DIR}/wikil2cc.json', 'r'))

wikil2cc['Q1860']['US']

94

In [92]:
from wiki_location import q2cc

US HR


In [96]:
def get_wiki_cc(country, birth_place, language):
    ccs = Counter()
    for bp in birth_place:
        if bp[5:] in q2cc:
            ccs[q2cc[bp[5:]]] += 3
    for cq in j['country']:
        if cq[5:] in q2cc:
            ccs[q2cc[cq[5:]]] += 1
    for lng in language:
        lng = lng[5:]
        if lng in wikil2cc:
            for cl,bod in wikil2cc[lng].items():
                ccs[cl] += bod / 100
    cc,_ = ccs.most_common()[0] if len(ccs)>0 else (None, None)
    return cc
    

In [110]:
uk = 0
fn = '/projekti/mondonomo/books/thai/thai_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('thai_', 'nonthai_'), 'w')
for i, l in enumerate(open('/backup/wikidata/wiki_person.jsonl')):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'th' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["th"][0].replace(',',' ')
        if cc == 'TH':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head /projekti/mondonomo/books/thai/thai_wiki.csv

Q45209,TH,Poonsawat Kratingdaenggym,พูนสวัสดิ์ กระทิงแดงยิม
Q52006,TH,Pen-Ek Karaket,เป็นเอก การะเกตุ
Q57670,TH,Yingluck Shinawatra,ยิ่งลักษณ์ ชินวัตร
Q74861,TH,Thanin Kraiwichian,ธานินทร์ กรัยวิเชียร
Q78455,TH,Phra Chenduriyang (Piti Wathayakon),พระเจนดุริยางค์ (ปิติ วาทยะกร)
Q158861,TH,Chulalongkorn,พระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว
Q243623,TH,Samak Sunthorawet,สมัคร สุนทรเวช
Q284211,TH,Prayad Boonya,ประหยัด บุญญา
Q301290,TH,Phraya Phahonphonphayuhasena (Phot Phahonyothin),พระยาพหลพลพยุหเสนา (พจน์ พหลโยธิน)
Q304412,TH,Narisara Nuwattiwong,สมเด็จพระเจ้าบรมวงศ์เธอ เจ้าฟ้ากรมพระยานริศรานุวัดติวงศ์


In [114]:
from tqdm.notebook import tqdm
uk = 0
fn = '/backup/wikidata/hr_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('hr_', 'nonhr_'), 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_571):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'hr' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["hr"][0].replace(',',' ')
        if cc == 'HR':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head $fn

  0%|          | 0/10044571 [00:00<?, ?it/s]

Q56220,HR,Ivo Sanader,Ivo Sanader
Q71774,HR,Igor Bubnjić,Igor Bubnjić
Q73376,HR,Wilhelm Ehm,Wilhelm Ehm
Q92358,HR,Tom Bunk,Tom Bunk
Q104440,HR,Vladimir Milojčić,Vladimir Milojčić
Q112081,HR,Archduke Wilhelm of Austria,Nadvojvoda Vilim Austrijski
Q114408,HR,Adolf Mussafia,Adolfo Mussafia
Q143424,HR,Vedran Mataija,Vedran Mataija
Q149090,HR,Ivan Ribar,Ivan Ribar
Q152506,HR,Nina Badrić,Nina Badrić


In [115]:
!wc -l  $fn

6627 /backup/wikidata/hr_wiki.csv
